# pytket-dqc Example Usage

In this notebook we gather some example uses of the pytket-dqc package.

## Networks

Near term networks of quantum servers are specified by two properties. The fist is the server coupling, detailing which servers are connected to which others. This is specified by a list of pairs of integers, where each pair signifies that there is a connection between those two servers. The second is the qubits each server contains. This is specified by a dictionary from the server to a list of qubits it contains. 

An example is given below, where blue lines indicate connections between servers, and red lines indicate connections between qubits within servers. The vertex labels are the indexes of the qubits.

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2], [0,3]], {0:[0,1,2,3], 1:[4,5,6], 2:[7,8,9,10,11], 3:[12,13]})
network.draw_nisq_network()

## Distributed Circuits

The `DistributedCircuits` class adds some additional properties and methods to the standard `Circuit` pytket class. These predominantly relate to a hypergraph representation of the circuit. `DistributedCircuits` are initialised from a standard tket `Circuit` object, as seen in the following example. Additional functionality is provided to visualise the corresponding hypergraph. For details on the correspondence between circuits and hypergraphs please see the paper [Automated distribution of quantum circuits via hypergraph partitioning](https://arxiv.org/abs/1811.10972).

In [ ]:
from pytket_dqc.circuits import DistributedCircuit
from pytket.circuit.display import render_circuit_jupyter
from pytket import Circuit

circ = Circuit(4).CZ(0,3).Rz(0.5,3).CZ(1,3).CZ(2,3).Rz(0.5,3)

dist_circ = DistributedCircuit(circ)
render_circuit_jupyter(dist_circ.circuit)
dist_circ.draw()

The accepted gate set for pytket-dqc is at present Rx, CZ, CX, Rz, Measure and QControlBox. QControlBox can be used to add controlled single qubit unitaries to your circuit, but nothing more complicated than that at the moment. Note that in the case of the hyperedges that relate to QControlBox, hyperedges are weighted. This is because they cannot be acted between servers in some directions using only one e-bit, and 2 may be required to perform a teleportation operation.

In [ ]:
from pytket.circuit import Circuit, CircBox, QControlBox, Op, OpType
from pytket.circuit.display import render_circuit_jupyter

op = Op.create(OpType.V)
cv = QControlBox(op, 1)

circ = Circuit(2)
circ.CZ(0,1)
circ.add_qcontrolbox(cv, [1,0]) 
circ.Rz(0.3,1)
circ.CZ(1,0)

dist_circ = DistributedCircuit(circ)
render_circuit_jupyter(dist_circ.circuit)
dist_circ.draw()

pytket-dqc includes some utilities for rebasing your circuit if it is not in the correct gate set initially.

In [ ]:
from pytket_dqc.utils import dqc_rebase

circ = Circuit(3).CY(0,1).CZ(1,2).H(1).CX(1,0)
render_circuit_jupyter(circ)

dqc_rebase.apply(circ)
render_circuit_jupyter(circ)

## Distributors

pytket-dqc provides several distributors, which are themselves a selection of methods to assign qubits and gates to servers. Their aim is to return a placement which minimises the e-bit cost of the resulting implementation.

Here we will work though some example use of these distributors. To do so let's define a simple network.

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2]], {0:[0], 1:[1,2], 2:[3,4]})
network.draw_nisq_network()

Let's also define a circuit to distribute. Some classes of circuits are predefined within pytket-dqc. Cyclic circuits, where CZ gates act in a circle, are one such class of circuits. These circuits are defined by the number of qubits they act on and the number of layers of cycles.

In [ ]:
from pytket_dqc.circuits import CyclicDistributedCircuit

dist_circ = CyclicDistributedCircuit(4,2)
render_circuit_jupyter(dist_circ.circuit)
dist_circ.draw()

One such is Brute, which performs a brutefoce search of all placements of qubits and gates onto servers, returning the one with the lowest cost. It is the slowest method, but returns the best result every time.

In [ ]:
from pytket_dqc.distributors import Brute
import time

distributor = Brute()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

Annealing is another approach, which uses simulated annealing as a means to arrive at a valid placement.

In [ ]:
from pytket_dqc.distributors import Annealing

distributor = Annealing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

GraphPartitioning uses the [Karlsruhe Hypergraph Partitioning Framework](https://kahypar.org/) to derive a placement.

In [ ]:
from pytket_dqc.distributors import GraphPartitioning

distributor = GraphPartitioning()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)

The placements resulting from using the GraphPartitioning method may not be valid on near term networks, as it does not take into consideration limits on the numbers of qubits in each server. It attempts to meet a load balancing condition, which aims to evenly balance the number of gates and qubits that each server must manage. This is not appropriate when the resources themselves are not evenly balanced.

In [ ]:
if placement.is_placement(dist_circ, network):
    print("final placement cost", placement.cost(dist_circ, network))
else:
    print("Unfortunatly this is not a valid placement.")

Routing makes use of routing and placement and routing techniques available in TKET. Here the network architecture as a whole is interpreted as a backend architecture, with noise on edges between servers set to be very high. Routing is guaranteed to generate a valid placement, and often very quickly. Unfortunately, it does not do a great job of distinguishing between connections within servers and connections between them. This can result in a high e-bit cost. Routing will also alter the circuit

In [ ]:
from pytket_dqc.distributors import Routing

distributor = Routing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

Note that it is possible to distribute circuits with controlled unitaries. This feature is under development, and may not work for all distributors. However...

In [ ]:
network = NISQNetwork([[0,1]], {0:[0], 1:[1]})
network.draw_nisq_network()

op = Op.create(OpType.V)
cv = QControlBox(op, 1)

circ = Circuit(2)
circ.add_qcontrolbox(cv, [0,1]) 
circ.add_qcontrolbox(cv, [0,1]) 
circ.Rz(0.3,0)
circ.add_qcontrolbox(cv, [0,1]) 
circ.Rx(0.3,0)
circ.add_qcontrolbox(cv, [0,1]) 
circ.add_qcontrolbox(cv, [0,1]) 

render_circuit_jupyter(circ)

dist_circ = DistributedCircuit(circ)
dist_circ.draw()

distributor = Brute()

placement = distributor.distribute(dist_circ, network)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

## Larger Example

Let's see how far we can push these schemes. Let's create a larger network:

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2], [0,3]], {0:[i for i in range(10)], 1:[i for i in range(10, 20)], 2:[i for i in range(20,30)], 3:[i for i in range(30,40)]})
network.draw_nisq_network()

and a larger circuit:

In [ ]:
from pytket_dqc.circuits import CyclicDistributedCircuit
from pytket.circuit.display import render_circuit_jupyter

dist_circ = CyclicDistributedCircuit(35,2)
render_circuit_jupyter(dist_circ.circuit)
dist_circ.draw()

We have already seen that Brute can be quite slow, so let's not use that here. Let's see how the others perform, starting with Annealing. In this case we see that it performs the slowest of the three schemes. However it does not take an unreasonably long time, and the solution is reasonably good.

In [ ]:
from pytket_dqc.distributors import Annealing
import time

distributor = Annealing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

GraphPartitioning is the quickest, and produces the best result. In this case this is not unsurprising. The servers do have balanced resources, and so we expect a valid solution in this case. The implementation of kahypar is highly optimised and so the result is arrived at very quickly.

In [ ]:
from pytket_dqc.distributors import GraphPartitioning

distributor = GraphPartitioning()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)

if placement.is_placement(dist_circ, network):
    print("final placement cost", placement.cost(dist_circ, network))
else:
    print("Unfortunatly this is not a valid placement.")

The placement cost of Routing is the worst, although this is not unreasonably so. Again it arrives a the solution very quickly.

In [ ]:
from pytket_dqc.distributors import Routing

distributor = Routing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

Let's finally take a more complex network:

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2], [2,3]], {0:[i for i in range(5)], 1:[i for i in range(5, 20)], 2:[i for i in range(20,30)], 3:[i for i in range(30,45)]})
network.draw_nisq_network()

and circuit:

In [ ]:
from pytket_dqc.circuits import RegularGraphDistributedCircuit
from pytket.circuit.display import render_circuit_jupyter

dist_circ = RegularGraphDistributedCircuit(34,3,5)
render_circuit_jupyter(dist_circ.circuit)
dist_circ.draw()

Annealing now takes a non-negligible time to find a placement. However, the solution is valid, and of those that generate a valid solution, it is the best

In [ ]:
from pytket_dqc.distributors import Annealing
import time

distributor = Annealing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))

GraphPartitioning is again remarkably quick. However, as the loads are unbalanced in this case, the result is not valid.

In [ ]:
from pytket_dqc.distributors import GraphPartitioning

distributor = GraphPartitioning()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)

if placement.is_placement(dist_circ, network):
    print("final placement cost", placement.cost(dist_circ, network))
else:
    print("Unfortunatly this is not a valid placement.")

Routing too is now noticeably slow, and the solution quite poor.

In [ ]:
from pytket_dqc.distributors import Routing

distributor = Routing()

start = time.time()
placement = distributor.distribute(dist_circ, network)
print("time to distribute", time.time() - start)
print("final placement", placement.placement)
print("final placement cost", placement.cost(dist_circ, network))